In [1]:
import numpy as np
import logging
import nibabel as nib
import math
from skimage.segmentation import slic
from pathlib import Path
from tqdm import tqdm

In [ ]:
data_dir = Path('/home/tony/Documents/IBSR_preprocessed/')
output_dir = Path('/home/tony/Documents/IBSR_preprocessed_2d/')
paths = [path for path in data_dir.iterdir() if path.is_dir()]

In [ ]:
def SLIC_transform(img, _n_segments, _compactness):
    #img = max_min_normalize(img)
    img = np.expand_dims(img, axis=2)
    img = np.concatenate((img, img, img), 2)
    segments = slic(img.astype('double'), n_segments=_n_segments, compactness=_compactness)
    return segments


def feature_extract(img, segments, f_range, n_vertex):
    #img = max_min_normalize(img)
    step = math.floor(f_range / 2)
    n_segments = np.max(segments) + 1
    features = np.zeros((n_vertex, f_range*f_range))
    #centroid_list = []
    for i in range(n_vertex):
        if i < n_segments:
            h_mean = int(np.where(segments==i)[0].mean())
            w_mean = int(np.where(segments==i)[1].mean())
            #centroid_list.append((h_mean, w_mean))
        else:
            h_mean = int(np.where(segments==0)[0].mean())
            w_mean = int(np.where(segments==0)[1].mean())
            #centroid_list.append((h_mean, w_mean))
        h = h_mean
        w = w_mean
        features[i] = img[h-step:h+step+1, w-step:w+step+1].flatten()
    return features

def max_min_normalize(img):
    img = np.asarray(img)
    return (img-np.min(img)) / (np.max(img)-np.min(img))

In [ ]:
for path in paths:
    #logging.info(f'Process {path.parts[-1]}.')
    print(f'Process {path.parts[-1]}.')
    # Create output directory
    if not (output_dir / path.parts[-1]).is_dir():
        (output_dir / path.parts[-1]).mkdir(parents=True)

    # Read in the MRI scans
    image = nib.load(str(path / 'image.nii.gz')).get_data().astype(np.float32)
    image = np.squeeze(image, 3)
    image = image.transpose(2, 0, 1)
    label = nib.load(str(path / 'label.nii.gz')).get_data().astype(np.uint8)
    label = np.squeeze(label, 3)
    label = label.transpose(2, 0, 1)

    # Save each slice of the scan into single file
    for s in tqdm(range(image.shape[0])):
        _image = image[s]
        _image = max_min_normalize(_image)
        segments = SLIC_transform(_image, n_segments, compactness)
        features, adj_arr = feature_extract(_image, segments, feature_range, n_vertex, tao)
    
        
        _label = label[s]
        label_new = label_transform(_label, segments)
        
        np.save((output_dir / path.parts[-1] / f'segments_{s}.npy'), segments)
        np.save((output_dir / path.parts[-1] / f'features_{s}.npy'), features)
        np.save((output_dir / path.parts[-1] / f'adj_arr_{s}.npy'), adj_arr)
        np.save((output_dir / path.parts[-1] / f'label_new_{s}.npy'), label_new)

In [2]:
image = nib.load('/home/tony/Documents/IBSR_preprocessed_2d/IBSR_01/image_80.nii.gz').get_data().astype(np.float32)
label = nib.load('/home/tony/Documents/IBSR_preprocessed_2d/IBSR_01/label_80.nii.gz').get_data().astype(np.int64)